# Demo For the GoWorkshop

## Linear Regression

### first we need to load the data
For this we will be using dataframe-go Library

In [19]:
import (
	"context"
	"fmt"
	"io/ioutil"
	"log"
	"strings"
	"github.com/rocketlaunchr/dataframe-go"
	"github.com/rocketlaunchr/dataframe-go/imports"
)

var ctx = context.Background()

func loadGopherData() *dataframe.DataFrame {
	content, err := ioutil.ReadFile("gopher_locations.csv")
	if err != nil {
		log.Fatal(err)
	}

	// Convert the content to a string
	csvString := string(content)
	df, err := imports.LoadFromCSV(ctx, strings.NewReader(csvString), imports.CSVLoadOptions{InferDataTypes: true, NilValue: &[]string{"NA"}[0]})
	if err != nil {
		panic(err)
	}

	return df
}

var df = loadGopherData()

In [20]:

func displayTheData(df *dataframe.DataFrame) {
 // Print the DataFrame
 fmt.Print(df.Table())
}

In [49]:
func main(){
    displayTheData(df)
	//Get the number of rows in the DataFrame
	numRows := df.NRows()

	// Get the first column by index
	X := df.Series[0]

	// Print the values of the first column
	for i := 0; i < int(numRows); i++ {
		value := X.Value(i)
		fmt.Println(value)
	}
}
    

+-------+----------------------+--------------------+-----------------+------------------+--------------+---------------+------------+----------+
|       |     LOCATIONNAME     | AVERAGETEMPERATURE | PREDATORDENSITY | FOODAVAILABILITY | WATERSOURCES | HUMANACTIVITY | VEGETATION | LANDAREA |
+-------+----------------------+--------------------+-----------------+------------------+--------------+---------------+------------+----------+
|  0:   |     Quiet Spring     |        -4.5        |     medium      |      medium      |     far      |     high      |   cliff    |   669    |
|  1:   |     Sunny Forest     |         23         |       low       |      medium      |     far      |   moderate    |   meadow   |   332    |
|  2:   |     Misty Dunes      |        38.8        |     medium      |       low        |     far      |     high      |   forest   |   942    |
|  3:   |    Bright Spring     |         23         |      high       |       high       |    close     |   moderate    |   

In [55]:
import (
	"context"
	"fmt"
	"io/ioutil"
	"log"
	"strconv"
	"strings"
	"github.com/gonum/stat"
	"gonum.org/v1/gonum/mat"
	"github.com/rocketlaunchr/dataframe-go"
	"github.com/rocketlaunchr/dataframe-go/imports"
)


func main() {

	// Extract the columns
	averageTemperature := df.Series[1]
	predatorDensity := df.Series[2]
	foodAvailability := df.Series[3]
	waterSources := df.Series[4]
	humanActivity := df.Series[5]
	vegetation := df.Series[6]
	landArea := df.Series[7]

	// Prepare X and y
	// For this example, let's use AverageTemperature as the target (y)
	// and PredatorDensity, FoodAvailability, WaterSources, HumanActivity, Vegetation, and LandArea as features (X)
	var X [][]float64
	var y []float64

	numRows := df.NRows()

	for i := 0; i < int(numRows); i++ {
		row := []float64{}

		// Convert categorical variables to numerical (simplified encoding)
		predatorDensityValue := map[string]float64{"low": 1, "medium": 2, "high": 3}[fmt.Sprintf("%v", predatorDensity.Value(i))]
		foodAvailabilityValue := map[string]float64{"low": 1, "medium": 2, "high": 3}[fmt.Sprintf("%v", foodAvailability.Value(i))]
		waterSourcesValue := map[string]float64{"near": 1, "medium": 2, "far": 3}[fmt.Sprintf("%v", waterSources.Value(i))]
		humanActivityValue := map[string]float64{"low": 1, "medium": 2, "high": 3}[fmt.Sprintf("%v", humanActivity.Value(i))]
		vegetationValue := map[string]float64{"forest": 1, "cliff": 2, "grassland": 3}[fmt.Sprintf("%v", vegetation.Value(i))]

		landAreaValue, err := strconv.ParseFloat(fmt.Sprintf("%v", landArea.Value(i)), 64)
		if err != nil {
			log.Fatalf("Error converting LandArea to float64: %v", err)
		}

		row = append(row, predatorDensityValue, foodAvailabilityValue, waterSourcesValue, humanActivityValue, vegetationValue, landAreaValue)
		X = append(X, row)

		averageTemperatureValue, err := strconv.ParseFloat(fmt.Sprintf("%v", averageTemperature.Value(i)), 64)
		if err != nil {
			log.Fatalf("Error converting AverageTemperature to float64: %v", err)
		}
		y = append(y, averageTemperatureValue)
	}

	// Convert X and y to matrices for gonum
	nSamples, nFeatures := len(X), len(X[0])
	Xmat := mat.NewDense(nSamples, nFeatures, nil)
	for i := range X {
		for j := range X[i] {
			Xmat.Set(i, j, X[i][j])
		}
	}

	ymat := mat.NewVecDense(nSamples, y)

	// Add a column of ones to X for the intercept term
	XWithIntercept := mat.NewDense(nSamples, nFeatures+1, nil)
	for i := 0; i < nSamples; i++ {
		XWithIntercept.Set(i, 0, 1) // intercept term
		for j := 0; j < nFeatures; j++ {
			XWithIntercept.Set(i, j+1, Xmat.At(i, j))
		}
	}

	// Compute (X^T * X)
	var XT mat.Dense
	XT.Mul(XWithIntercept.T(), XWithIntercept)

	// Compute (X^T * y)
	var XTy mat.VecDense
	XTy.MulVec(XWithIntercept.T(), ymat)

	// Solve for beta (coefficients)
	var beta mat.VecDense
	err := beta.SolveVec(&XT, &XTy)
	if err != nil {
		log.Fatal(err)
	}

	// Print the coefficients
	fmt.Printf("Coefficients: %v\n", mat.Formatted(&beta, mat.Prefix("             "), mat.Squeeze()))
}

Coefficients: ⎡  12.537097419536728⎤
             ⎢ 0.40869494110716387⎥
             ⎢-0.21431649816529333⎥
             ⎢-0.09737403641504794⎥
             ⎢  0.6476661883408436⎥
             ⎢ -0.8911282337266179⎥
             ⎣0.006035263591470038⎦


In [61]:
import (
    "github.com/sjwhitworth/golearn/base"
    "github.com/sjwhitworth/golearn/linear_model"
    )

func main(){
    dataGrid := base.ConvertDataFrameToInstances(df, 0)
    // Create a linear regression model
    model := linear_models.NewLinearRegression()

    // Fit the model to the data
    err = model.Fit(dataGrid)
    if err != nil {
        // Handle error
        return
    }
    
    
}

# github.com/sjwhitworth/golearn/linear_models 

 


 /opt/go/pkg/mod/github.com/sjwhitworth/golearn@v0.0.0-20221228163002-74ae077eafb2/linear_models/logistic.go:10:9: undefined: Parameter 

 


 /opt/go/pkg/mod/github.com/sjwhitworth/golearn@v0.0.0-20221228163002-74ae077eafb2/linear_models/logistic.go:11:9: undefined: Model 

 


 /opt/go/pkg/mod/github.com/sjwhitworth/golearn@v0.0.0-20221228163002-74ae077eafb2/linear_models/logistic.go:17:17: undefined: L2R_LR 

 


 /opt/go/pkg/mod/github.com/sjwhitworth/golearn@v0.0.0-20221228163002-74ae077eafb2/linear_models/logistic.go:19:17: undefined: L1R_LR 

 


 /opt/go/pkg/mod/github.com/sjwhitworth/golearn@v0.0.0-20221228163002-74ae077eafb2/linear_models/logistic.go:25:13: undefined: NewParameter 

 


 /opt/go/pkg/mod/github.com/sjwhitworth/golearn@v0.0.0-20221228163002-74ae077eafb2/linear_models/logistic.go:33:10: undefined: NewProblem 

 


 /opt/go/pkg/mod/github.com/sjwhitworth/golearn@v0.0.0-20221228163002-74ae077eafb2/linear_models/logistic.go:34:13: undefined: Train 

 


 /opt/go/pkg/mod/github.com/sjwhitworth/golearn@v0.0.0-20221228163002-74ae077eafb2/linear_models/logistic.go:58:10: undefined: Predict

ERROR: failed to run "/usr/local/go/bin/go build -o /tmp/gonb_8fe8148f/gonb_8fe8148f": exit status 1